### Import

In [5]:
import pandas as pd
import os

### Load Data

For the folder path you can write or use a function shared with you in the code

In [13]:
folder_path = "../ynov-data"

In [15]:
from ynov import utils

In [16]:
folder_path = utils.get_data_path()

In [18]:
print(folder_path)

/Users/FRANCOIS/project/trainings/YNOV/2425-YNOV/ynov-data


In [24]:
df_train = pd.read_csv(os.path.join(folder_path, "train_housing.csv"))
submission = pd.read_csv(os.path.join(folder_path, "submission.csv"))

### Random Split

In [25]:
from sklearn.model_selection import train_test_split

In [29]:
train_df, valid_df = train_test_split(df_train, test_size=0.3, random_state=42)

I recommend to save the new train and valid to use the same ones later

In [30]:
train_df.to_csv(os.path.join(folder_path, "train_housing_data.csv"))
valid_df.to_csv(os.path.join(folder_path, "valid_housing_data.csv"))

### Stratified split

In [31]:
df_train.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'id'],
      dtype='object')

In [34]:
num_bins = 10  # Number of bins
df_train['Target_bin'] = pd.qcut(df_train['median_house_value'], q=num_bins, labels=False)  #

In [35]:
train_df, valid_df = train_test_split(df_train, test_size=0.3, random_state=42, stratify=df_train['Target_bin'])

In [36]:
train_df.to_csv(os.path.join(folder_path, "train_housing_data_stratified.csv"))
valid_df.to_csv(os.path.join(folder_path, "valid_housing_data_stratified.csv"))

### Calculate mean on new train

In [38]:
average_median_house_value = train_df.median_house_value.mean()

In [39]:
print(average_median_house_value)

206609.7363095238


### Apply it to valid on calculate the RMSE

In [50]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [51]:
valid_df["predictions"] = average_median_house_value
train_df["predictions"] = average_median_house_value

In [52]:
rmse_train = root_mean_squared_error(train_df["predictions"], train_df["median_house_value"])
rmse_valid = root_mean_squared_error(valid_df["predictions"], valid_df["median_house_value"])

In [53]:
print(f"the RMSE on train: is {rmse_train}")

the RMSE on train: is 115690.43788022296


In [54]:
print(f"the RMSE on valid: is {rmse_valid}")

the RMSE on valid: is 115880.55613810578


### Build submission file 

In [58]:
submission["median_house_value"] = average_median_house_value

In [60]:
submission.to_csv(os.path.join(folder_path,"submissions","subission_baseline.csv"), sep=",", index=False)
